In [ ]:
# Wojciech Geisler
# 2018

In [2]:
autodiff(ex::Real) = :(0)
autodiff(ex::Symbol) = :(1)

function autodiff(ex::Expr)
    if(ex.head == :call)
        if(ex.args[1] isa GlobalRef)
            ex.args[1] = ex.args[1].name
        end
        autodiff(Val{ex.args[1]}, ex)
        
    else
        throw(DomainError())
    end
end



autodiff (generic function with 3 methods)

In [3]:

function autodiff(::Type{Val{:+}}, ex::Expr)
    diffs = map(autodiff, ex.args[2:end])
    res = diffs[1]
    for d in diffs[2:end]
        res = :($res + $d)
    end
    res
end

function autodiff(::Type{Val{:-}}, ex::Expr)
    l = autodiff(ex.args[2])
    r = autodiff(ex.args[3])
    :($l - $r)
end

function autodiff(::Type{Val{:*}}, ex::Expr)
    # result of multiplication differential is a sum where each
    # addend has on of the original arguments differentialed
    res = :(0)
    children = ex.args[2:end]
    diffs = map(autodiff, children)
    
    for i in 1:length(diffs)
        part = 1
        for j in 1:length(diffs)
            if i == j
                d = diffs[j]
                part = :($part * $d)
            else
                c = children[j]
                part = :($part * $c)
            end
        end
        res = :($res + $part)
    end
       
    
    res
end

function autodiff(::Type{Val{:/}}, ex::Expr)
    l = ex.args[2]
    r = ex.args[3]
    ldx = autodiff(l)
    rdx = autodiff(r)
    :(($ldx*$r - $l * $rdx) / ($r * $r))
end

autodiff (generic function with 7 methods)

In [4]:
e = autodiff(:(x * 5x * 2))

:(((0 + ((1 * 1) * (5x)) * 2) + ((1x) * ((0 + (1 * 0) * x) + (1 * 5) * 1)) * 2) + ((1x) * (5x)) * 0)

In [5]:
x = 3
eval(e)

60

In [6]:
# replace slotnumbers with symbols
function fillSlots(expr::SlotNumber, symbol)
    return symbol
end
function fillSlots(expr::Expr, symbol)
    expr.args = map(x -> fillSlots(x, symbol), expr.args)
    expr
end
function fillSlots(expr, symbol)
    expr
end

fillSlots (generic function with 3 methods)

In [11]:
function autodiffFun(f)
    code_info = code_lowered(f)
    code = code_info[1].code[2].args[1] # remove return operation
    expr = fillSlots(code, :_x)
    diff = autodiff(expr)
    return eval(:(_x -> $diff))
    
end

autodiffFun (generic function with 1 method)

In [12]:
f(x) = 0.5 * x * x * x + 2x
g = autodiffFun(f)
g(7)

75.5

In [14]:
h = autodiffFun(g)

(::#7) (generic function with 1 method)

In [15]:
h(2)

6.0